# Ollama + OpenAI + Python

## 1. مشخص کردن نام مدل

اگر مدل دیگری به جز "phi3:mini" را وارد کرده‌اید، مقدار موجود در سلول زیر را تغییر دهید. این متغیر در کدهای سراسر نوت‌بوک استفاده خواهد شد.


In [ ]:
MODEL_NAME = "phi3:mini"

## ۲. تنظیم کلاینت Open AI

به طور معمول کلاینت OpenAI برای تعامل با مدل‌های زبان بزرگ از OpenAI.com یا Azure OpenAI استفاده می‌شود.  
با این حال، می‌توان از آن با Ollama نیز استفاده کرد، زیرا Ollama یک نقطه پایانی سازگار با OpenAI در "http://localhost:11434/v1" ارائه می‌دهد.


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. تولید یک پاسخ چت

اکنون می‌توانیم از OpenAI SDK برای تولید پاسخ یک مکالمه استفاده کنیم. این درخواست باید یک هایکو درباره گربه‌ها ایجاد کند:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. مهندسی درخواست

اولین پیامی که به مدل زبان ارسال می‌شود "پیام سیستم" یا "درخواست سیستم" نامیده می‌شود و دستورالعمل‌های کلی برای مدل را تعیین می‌کند.  
شما می‌توانید درخواست سیستم خود را سفارشی کنید تا مدل زبان را به تولید خروجی به شیوه‌ای متفاوت هدایت کنید.  
`SYSTEM_MESSAGE` زیر را تغییر دهید تا مدل به سبک شخصیت مورد علاقه شما از یک فیلم یا سریال معروف پاسخ دهد، یا از درخواست‌های دیگر در [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts) الهام بگیرید.

پس از سفارشی کردن پیام سیستم، اولین سوال کاربر را در `USER_MESSAGE` ارائه دهید.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## ۵. مثال‌های محدود

یکی دیگر از روش‌های هدایت مدل زبان، ارائه "چند مثال" است، یعنی یک سری سوال/پاسخ نمونه که نشان می‌دهد مدل چگونه باید پاسخ دهد.

مثال زیر تلاش می‌کند تا مدل زبان را به عنوان یک دستیار آموزشی شبیه‌سازی کند، با ارائه چند نمونه از سوالات و پاسخ‌هایی که ممکن است یک دستیار آموزشی بدهد، و سپس مدل را با سوالی که ممکن است یک دانش‌آموز بپرسد، تحریک می‌کند.

ابتدا آن را امتحان کنید، سپس `SYSTEM_MESSAGE`، `EXAMPLES` و `USER_MESSAGE` را برای یک سناریوی جدید تغییر دهید.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. تولید مبتنی بر بازیابی

RAG (تولید مبتنی بر بازیابی) یک تکنیک است که به مدل‌های زبانی کمک می‌کند تا به سوالات مربوط به یک حوزه خاص با دقت پاسخ دهند. این کار با بازیابی اطلاعات مرتبط از یک منبع دانش و سپس تولید پاسخ بر اساس آن اطلاعات انجام می‌شود.

ما یک فایل CSV محلی با داده‌هایی درباره خودروهای هیبریدی فراهم کرده‌ایم. کد زیر فایل CSV را می‌خواند، به دنبال تطابق‌هایی با سوال کاربر می‌گردد و سپس بر اساس اطلاعات یافت‌شده پاسخ تولید می‌کند. توجه داشته باشید که این فرآیند نسبت به مثال‌های قبلی زمان بیشتری می‌برد، زیرا داده‌های بیشتری به مدل ارسال می‌شود. اگر متوجه شدید که پاسخ هنوز بر اساس داده‌ها نیست، می‌توانید مهندسی سیستم را امتحان کنید یا از مدل‌های دیگر استفاده کنید. به طور کلی، RAG با مدل‌های بزرگ‌تر یا نسخه‌های تنظیم‌شده SLM‌ها مؤثرتر است.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
